# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("data/song_data")

In [5]:
filepath = song_files[0]

In [6]:
df = pd.read_json(filepath, lines = True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
df[['song_id','title','artist_id','year','duration']].values

array([['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0,
        218.93179]], dtype=object)

In [8]:
song_data = df[['song_id','title','artist_id','year','duration']].values
song_data = song_data[0].tolist()

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values
artist_data = artist_data[0].tolist()

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [12]:
log_files = get_files("data/log_data")

In [13]:
filepath = log_files[0]

In [14]:
df = pd.read_json(filepath, lines = True)
df.head().T

,0,1,2,3,4
artist,Stephen Lynch,Manowar,Morcheeba,Maroon 5,Train
auth,Logged In,Logged In,Logged In,Logged In,Logged In
firstName,Jayden,Jacob,Jacob,Jacob,Jacob
gender,M,M,M,M,M
itemInSession,0,0,1,2,3
lastName,Bell,Klein,Klein,Klein,Klein
length,182.857,247.562,257.41,231.235,216.764
level,free,paid,paid,paid,paid
location,"Dallas-Fort Worth-Arlington, TX","Tampa-St. Petersburg-Clearwater, FL","Tampa-St. Petersburg-Clearwater, FL","Tampa-St. Petersburg-Clearwater, FL","Tampa-St. Petersburg-Clearwater, FL"
method,PUT,PUT,PUT,PUT,PUT


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [15]:
df = df.loc[df["page"] == "NextSong"]
df.head().T

,0,1,2,3,4
artist,Stephen Lynch,Manowar,Morcheeba,Maroon 5,Train
auth,Logged In,Logged In,Logged In,Logged In,Logged In
firstName,Jayden,Jacob,Jacob,Jacob,Jacob
gender,M,M,M,M,M
itemInSession,0,0,1,2,3
lastName,Bell,Klein,Klein,Klein,Klein
length,182.857,247.562,257.41,231.235,216.764
level,free,paid,paid,paid,paid
location,"Dallas-Fort Worth-Arlington, TX","Tampa-St. Petersburg-Clearwater, FL","Tampa-St. Petersburg-Clearwater, FL","Tampa-St. Petersburg-Clearwater, FL","Tampa-St. Petersburg-Clearwater, FL"
method,PUT,PUT,PUT,PUT,PUT


In [16]:
df['nts'] =  pd.to_datetime(df['ts'], unit='ms')

In [17]:
t = df.copy()
t['year'] = pd.DatetimeIndex(df['nts']).year
t['month'] = pd.DatetimeIndex(df['nts']).month
t['week'] = pd.DatetimeIndex(df['nts']).week
t['day'] = pd.DatetimeIndex(df['nts']).day
t['hour'] = pd.DatetimeIndex(df['nts']).hour
t['week_day'] = pd.DatetimeIndex(df['nts']).weekday_name
t.head().T

,0,1,2,3,4
artist,Stephen Lynch,Manowar,Morcheeba,Maroon 5,Train
auth,Logged In,Logged In,Logged In,Logged In,Logged In
firstName,Jayden,Jacob,Jacob,Jacob,Jacob
gender,M,M,M,M,M
itemInSession,0,0,1,2,3
lastName,Bell,Klein,Klein,Klein,Klein
length,182.857,247.562,257.41,231.235,216.764
level,free,paid,paid,paid,paid
location,"Dallas-Fort Worth-Arlington, TX","Tampa-St. Petersburg-Clearwater, FL","Tampa-St. Petersburg-Clearwater, FL","Tampa-St. Petersburg-Clearwater, FL","Tampa-St. Petersburg-Clearwater, FL"
method,PUT,PUT,PUT,PUT,PUT


In [18]:
time_data = ('nts', 'hour', 'day', 'week', 'month', 'year', 'week_day')
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [19]:
time_df = t[['nts', 'hour', 'day', 'week', 'month', 'year', 'week_day']]
time_df.head()

,nts,hour,day,week,month,year,week_day
0,2018-11-30 00:22:07.796,0,30,48,11,2018,Friday
1,2018-11-30 01:08:41.796,1,30,48,11,2018,Friday
2,2018-11-30 01:12:48.796,1,30,48,11,2018,Friday
3,2018-11-30 01:17:05.796,1,30,48,11,2018,Friday
4,2018-11-30 01:20:56.796,1,30,48,11,2018,Friday


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [20]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [21]:
user_df = t[['userId', 'firstName', 'lastName', 'gender', 'level']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [22]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [23]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.nts, row.userId, row.level, songid, artistid, row.itemInSession, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [24]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.